In [17]:
from IPython.display import clear_output
import numpy as np

In [14]:
DATA_COUNT = 1000
DIM = 4
DATA_LIMIT = 1

Rotation stuffs

In [23]:
def rotate_vec(dim=4, theta=None, i=None, j=None):

    if theta is None:
        theta = 2 * np.pi * np.random.random()

    if i is None:
        i = np.random.choice(dim)

    if j is None:
        j = np.random.choice(dim)
        while i == j:
            j = np.random.choice(dim)

    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return rotational_matrix


def random_rot(dim, count=10):
    id = np.identity(dim)
    for i in range(count):
        id = id.dot(rotate_vec(dim=dim))
    return id


Creating data

In [24]:


def tensor_diff(a, b):
    c = a - b
    c = np.squeeze(c)
    c_t = np.var(c)
    c = np.square(c)
    c = np.sqrt(np.average(c))
    return np.average(c), c_t, np.exp(- np.average(c)) * 100

def dropout_p_matrix(percent,shp, step = True):
    dropout_p = np.random.random(shp)
    d_l = np.vectorize(lambda x : 1 if x < percent else 0)
    if step:
        return(d_l(dropout_p))
    else:
        return dropout_p

def hyp(vex, rotational_matrix):
    return vex.dot(rotational_matrix)

In [25]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [21]:

def regressutil(weights, training_data, training_label, epochs, testing_data, testing_label,alpha = 0.001, update_interval = 10, retain = 1):
    for epoch in range(epochs):
        predicted_label = training_data.dot(weights * dropout_p_matrix(retain, weights.shape))
        
        dW = - (2 * training_data.T.dot(training_label - predicted_label)) / training_data.size
        weights -= dW * alpha

        if epoch % (epochs // update_interval) == 0:
            clear_output()
            print("Error :", tensor_diff(testing_data.dot(weights), testing_label))
            print("Weight")
            print(weights)
    return weights

        

In [22]:
def regress(W, vecs, vecsy, epochs):
    vecs = np.array(np.split(vecs, 10))
    vecsy = np.array(np.split(vecsy, 10))
    for i in range(10):
        W = regressutil(W, np.vstack(np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0)), epochs //10, vecs[i], vecsy[i], update_interval = 1, retain = (i/30 + 0.7))

    return W

In [32]:
W = np.random.random((DIM, DIM))
vecs = (np.random.random((DATA_COUNT, DIM)) * 2 * DATA_LIMIT) - DATA_LIMIT
r = random_rot(DIM)
vecsy = hyp(vecs, r)

In [33]:
r

array([[ 0.7787, -0.4367, -0.4383, -0.1038],
       [ 0.    ,  0.7084, -0.7058,  0.    ],
       [-0.4507, -0.3082, -0.3094, -0.7785],
       [ 0.4364,  0.4609,  0.4626, -0.6189]])

In [ ]:
regress(W, vecs, vecsy, 800000)

In [34]:
regressutil(W, vecs, vecsy, 80000, vecs, vecsy, update_interval = 10, retain = 1, alpha= 0.001)

Error : (4.9327606601431634e-06, 2.4284276499430964e-11, 99.99950672515058)
Weight
[[ 0.7787 -0.4367 -0.4383 -0.1038]
 [ 0.      0.7084 -0.7058  0.    ]
 [-0.4507 -0.3082 -0.3094 -0.7785]
 [ 0.4364  0.4609  0.4627 -0.6189]]


array([[ 0.7787, -0.4367, -0.4383, -0.1038],
       [ 0.    ,  0.7084, -0.7058,  0.    ],
       [-0.4507, -0.3082, -0.3094, -0.7785],
       [ 0.4364,  0.4609,  0.4627, -0.6189]])